In [2]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para procesamiento de texto
import re
import string
import langdetect
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

# Extracción de palabras clave
import yake
from rake_nltk import Rake

# Similitud semántica
from sentence_transformers import SentenceTransformer, util

# Visualización y análisis
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Modelado y evaluación
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Utilidades
from tqdm import tqdm
import os

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jhonr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
# Ruta del archivo de datos
ruta_archivo = r"C:\Users\Jhonr\Project_ODS_BERT\Data_total\Dataset_Inferencia.xlsx"

# Carga del archivo Excel
df = pd.read_excel(ruta_archivo)

# Visualización inicial de las primeras filas
print("Vista preliminar del archivo:")
print(df.head())

# Verificación de columnas disponibles
print("\nColumnas encontradas en el archivo:")
print(df.columns.tolist())

# Verificación de valores nulos por columna
print("\nConteo de valores nulos por columna:")
print(df.isnull().sum())

# Filtrado de registros con resumen válido
df = df[df['Abstract'].notnull() & (df['Abstract'].str.strip() != '')]

# Revisión de registros después del filtrado
print(f"\nTotal de resúmenes válidos encontrados: {len(df)}")


Vista preliminar del archivo:
                                             Authors      Area  \
0  Guevara-Cuellar C.A.; Parody-Rúa E.; Garcia-Pe...  Medicina   
1  Fernández-Niño J.A.; Bonilla-Tinoco L.J.; Manr...  Medicina   
2      Álvarez A.; Jiménez Á.; Méndez J.; Murillo E.  Medicina   
3           Giraldo J.; Acosta C.; Giraldo-Grueso M.  Medicina   
4  Cardozo D.F.R.; Martinez W.J.A.; Lopez J.F.C.;...  Medicina   

                                   Author full names  \
0  Guevara-Cuellar, César Augusto (23489087300); ...   
1  Fernández-Niño, Julián Alfredo (20433473700); ...   
2  Álvarez, Andree (59239404100); Jiménez, Ángel ...   
3  Giraldo, Juan (7003797713); Acosta, Claudia (5...   
4  Cardozo, Diego F Rincon (57194755896); Martine...   

                                               Title  Year  \
0  Cost Effectiveness of Combination Therapy Vers...  2018   
1  Work status, retirement, and depression in old...  2018   
2  Chemical and biological study of Eugenia Stipi.

In [4]:
# Limpieza específica para YAKE (basada en Yake_mejor_test_6-7-13.ipynb)
def limpiar_texto_yake(texto):
    texto = str(texto).strip()
    texto = texto.lower()
    return texto

# Limpieza específica para BERT (basada en GPU_Area_Clases6_7_9_13_OTROS_GPT.ipynb)
def limpiar_texto_bert(texto):
    texto = BeautifulSoup(texto, "html.parser").get_text()
    texto = re.sub(r'\n+', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[^\w\s]', '', texto)
    return texto.strip()

# Aplicar a columnas
df["abstract_yake"] = df["Abstract"].apply(limpiar_texto_yake)
df["abstract_bert"] = df["Abstract"].apply(limpiar_texto_bert)

# Mostrar ejemplo
print("\nEjemplo de resumen limpio para YAKE:")
print(df["abstract_yake"].iloc[0])

print("\nEjemplo de resumen limpio para BERT:")
print(df["abstract_bert"].iloc[0])


C:\Users\Jhonr\AppData\Local\Temp\ipykernel_24476\4261797259.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texto = BeautifulSoup(texto, "html.parser").get_text()



Ejemplo de resumen limpio para YAKE:
objectives: to estimate the incremental cost effectiveness ratio of pharmacological treatment for benign prostatic hyperplasia from the payer's perspective. methods: the cost effectiveness of 5 mg finasteride, 0.5 mg dutasteride, 10 mg alfuzosin, 10 mg terazosin, 0.4 mg tamsulosin, 4 mg doxazosin, and the combination therapy of 5 mg finasteride and 8 mg doxazosin was evaluated using a markov model over a 30-year period. the costs were estimated using national tariffs and were reported in us dollars. cost and effectiveness outcomes were discounted at a rate of 5% per year. men (aged ≥40 years) with moderate to severe lower urinary tract symptoms and uncomplicated benign prostatic hyperplasia were included in the analysis. outcomes included costs and quality-adjusted life-years. a probabilistic sensitivity analysis was performed on important parameters with monte-carlo simulation. results: finasteride alone or in combination with doxazosin dominated 

FASE 1 – YAKE (ODS 6, 7 y 13)

Paso 1: Definir diccionario de keywords por ODS

In [5]:
# Diccionario de palabras clave representativas por ODS
keywords_ods = {
    "ODS 6": ["water", "sanitation", "wastewater", "hygiene", "clean", "access", "drinking", "sewage", "treatment", "infrastructure"],
    "ODS 7": ["energy", "renewable", "solar", "electricity", "clean", "affordable", "efficiency", "power", "grid", "supply"],
    "ODS 13": ["climate", "emissions", "carbon", "resilience", "adaptation", "warming", "sustainability", "mitigation", "ghg", "environment"]
}


Paso 2: Inicializar YAKE y función para obtener keywords

In [6]:
kw_extractor = yake.KeywordExtractor(lan="en", n=1, top=10)

def obtener_keywords_yake(texto):
    try:
        keywords = kw_extractor.extract_keywords(texto)
        return [kw for kw, score in keywords]
    except:
        return []


Paso 3: Calcular intersección con listas ODS

In [7]:
def asignar_ods_por_yake(keywords_extraidas):
    max_match = 0
    mejor_ods = None
    for ods, lista_ods in keywords_ods.items():
        coincidencias = len(set(keywords_extraidas).intersection(set(lista_ods)))
        if coincidencias > max_match:
            max_match = coincidencias
            mejor_ods = ods
    return mejor_ods if max_match >= 1 else None  # Umbral mínimo de coincidencia = 1


Paso 4: Aplicar YAKE a todos los resúmenes

In [8]:
# Extraer keywords y predecir ODS
df["keywords_yake"] = df["abstract_yake"].apply(obtener_keywords_yake)
df["prediccion_yake"] = df["keywords_yake"].apply(asignar_ods_por_yake)

# Revisión rápida
print("\nDistribución de predicciones YAKE:")
print(df["prediccion_yake"].value_counts(dropna=False))



Distribución de predicciones YAKE:
prediccion_yake
None      2632
ODS 7      151
ODS 6      141
ODS 13      47
Name: count, dtype: int64


Código base para la inferencia con BERT

 1. Filtro y separación por área

In [9]:
# Filtrar resúmenes no clasificados por YAKE
df_bert = df[df["prediccion_yake"].isna()].copy()

# Dividir por área
df_med = df_bert[df_bert["Area"] == "Medicina"].copy()
df_ing = df_bert[df_bert["Area"] == "Ingeniería"].copy()


2. Preparar clase para inferencia con HuggingFace

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from torch.utils.data import Dataset

# Clase personalizada de Dataset para HuggingFace
class ODSDataset(Dataset):
    def __init__(self, textos, tokenizer, max_length=256):
        self.encodings = tokenizer(textos, truncation=True, padding="max_length", max_length=max_length)

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


 3. Función para cargar modelo y predecir

In [11]:
def inferir_con_bert(df_area, ruta_modelo):
    tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)
    model = AutoModelForSequenceClassification.from_pretrained(ruta_modelo)

    textos = df_area["abstract_bert"].tolist()
    dataset = ODSDataset(textos, tokenizer)

    trainer = Trainer(model=model)
    predicciones = trainer.predict(dataset)
    
    labels_pred = torch.argmax(torch.tensor(predicciones.predictions), axis=1).tolist()

    # Mapear ID de clase a nombre
    id2label = {0: "ODS 3", 1: "ODS 9", 2: "OTROS"}
    etiquetas = [id2label[i] for i in labels_pred]

    df_area["prediccion_bert"] = etiquetas
    return df_area


4. Ejecutar inferencia por área

In [12]:
# Rutas a tus modelos entrenados
ruta_modelo_med = r"C:\Users\Jhonr\Project_ODS_BERT\Modelos\Medicina"
ruta_modelo_ing = r"C:\Users\Jhonr\Project_ODS_BERT\Modelos\Ingenieria"

# Aplicar a cada subconjunto
df_med_resultado = inferir_con_bert(df_med, ruta_modelo_med)
df_ing_resultado = inferir_con_bert(df_ing, ruta_modelo_ing)

# Unir resultados
df_bert_final = pd.concat([df_med_resultado, df_ing_resultado])


OSError: Incorrect path_or_model_id: 'C:\Users\Jhonr\Project_ODS_BERT\Modelos\Medicina'. Please provide either the path to a local folder or the repo_id of a model on the Hub.